In [2]:
import pandas as pd
import numpy as np
import os
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import re
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
nltk.download("stopwords")
stope_words_set = set(stopwords.words("english"))
from nltk.tokenize import word_tokenize

def remove_stop_words(summary):
    flag = 0
    summary_word_token = word_tokenize(summary)
    updated_summary = ""
    for w in summary_word_token:
        if w not in stope_words_set:
            if flag == 1:
                updated_summary = updated_summary + " " + w
            else:
                updated_summary = w
                flag = flag +1
    #print(updated_summary)
    return updated_summary

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aditi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
def input_encoding(url):
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'Referer': 'https://www.google.com/',
}

    response = requests.get(url, headers=headers , timeout=15)
    response.raise_for_status()

    sample_soup = BeautifulSoup(response.text, 'html.parser')
    
    encoded = []
    
    encoded.append(sample_soup.find('span',{'class':'sc-eb51e184-1 cxhhrI'}).text)
    
    year_duration = sample_soup.find('ul',{'class' : 'ipc-inline-list ipc-inline-list--show-dividers sc-d8941411-2 cdJsTz baseAlt'})
    year_duration = year_duration.find_all('li',{'class':'ipc-inline-list__item'})
    flag=0
    for item in year_duration:
        if flag == 0:
            sample_year=item.text
            encoded.append(sample_year)
            flag = flag+1
            
    sample_genre=[]
    sample_genre_tag = sample_soup.find('div',{'class':'ipc-chip-list__scroller'})
    sample_genre_tag = sample_genre_tag.find_all('a',{'class':'ipc-chip ipc-chip--on-baseAlt'})
    for item in sample_genre_tag:
        g = item.find('span',{'class':'ipc-chip__text'}).text
        sample_genre.append(g)
    encoded.append(sample_genre[0])
    
    
    sample_intro = sample_soup.find('p' , {'class' : 'sc-2d37a7c7-3 dmZChI'})
    sample_intro = sample_intro.find('span' , {'class' : 'sc-2d37a7c7-0 caYjFF'}).text
    encoded.append(remove_stop_words(sample_intro))
    
    
    input_string = ""
    flag=0
    for item in encoded:
        if flag == 0:
            input_string = item
            flag = flag +1
        else:
            input_string = input_string + " " + item
            
    return input_string
            
def vectorization(input_string):
    input_series = pd.Series(input_string)
    vectorize = TfidfVectorizer(stop_words="english")
    vectorize_matrix = vectorize.fit_transform(input_string)
    return vectorize_matrix
            
            
def find_similar_movies(vectorize_matrix, input_vector, movies_name, top_n=5):
    cosine_sim = cosine_similarity(input_vector, vectorize_matrix)
    similar_indices = cosine_sim.argsort().flatten()[-top_n:]
    similar_movies = [movies_name[idx] for idx in similar_indices]
    return similar_movies
        
    

In [13]:
with open(r'C:\Users\aditi\OneDrive\Desktop\PROJECTS\Movie-Recommendation-System\encoded_movies_info.txt', 'rb') as file:
    encoded_vector = pickle.load(file,encoding='utf-8')
with open(r'C:\Users\aditi\OneDrive\Desktop\PROJECTS\Movie-Recommendation-System\movies_name.txt', 'rb') as file:
    movies_name = pickle.load(file , encoding='utf-8') 

In [14]:
url = "https://www.imdb.com/title/tt8672856/"

sample_movie = input_encoding(url)
sample_vector = vectorization(sample_movie)
print(find_similar_movies(sample_vector,encoded_vector))

ValueError: Iterable over raw text documents expected, string object received.